In [0]:
from google.colab import drive

In [13]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# ls
import os
os.chdir('/content/drive/My Drive/cnn_project')

In [15]:
ls

plant-seedlings-classification.zip


In [0]:
# 1. Read the images and generate the train and test dataset (5 points)
from zipfile import ZipFile
with ZipFile('plant-seedlings-classification.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile('train.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile('test.zip', 'r') as z:
  z.extractall()

In [0]:
# 2.  Divide the data set into Train and validation data sets


os.chdir('/content/drive/My Drive/cnn_project/train')

In [23]:
ls

 Black-grass/        'Common wheat'/      'Scentless Mayweed'/
 Charlock/           'Fat Hen'/           'Shepherds Purse'/
 Cleavers/           'Loose Silky-bent'/  'Small-flowered Cranesbill'/
'Common Chickweed'/   Maize/              'Sugar beet'/


In [24]:
x_train = []
y_train = []
import cv2
for i in os.listdir():
    print('+++')
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread('/content/drive/My Drive/cnn_project/train/' + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128))
                    x_train.append(dummy)
                    y_train.append(i)
                except Exception as e:
                    print(e)

+++
Fat Hen
+++
Small-flowered Cranesbill
+++
Cleavers
+++
Black-grass
+++
Sugar beet
+++
Shepherds Purse
+++
Charlock
+++
Loose Silky-bent
+++
Scentless Mayweed
+++
Maize
+++
Common Chickweed
+++
Common wheat


In [34]:

print(x_train[0].shape)
print(len(y_train))
print(len(x_train))

(128, 128, 3)
4750
4750


In [0]:
import pandas as pd
y_train = pd.get_dummies(y_train)

In [37]:
y_train.head()

,Black-grass,Charlock,Cleavers,Common Chickweed,Common wheat,Fat Hen,Loose Silky-bent,Maize,Scentless Mayweed,Shepherds Purse,Small-flowered Cranesbill,Sugar beet
0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0


In [0]:
import numpy as np
y_train = np.array(y_train)

In [40]:
y_train[0]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [44]:
type(x_train)
import numpy as np
x_train = np.array(x_train)
type(x_train)

numpy.ndarray

In [45]:
type(y_train)

numpy.ndarray

In [47]:
x_train[0].shape

(128, 128, 3)

In [0]:
from sklearn.model_selection import train_test_split
x_train_set, x_test, y_train_set, y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=10)


In [49]:
print(x_train_set[0].shape)
print(x_test[0].shape)

(128, 128, 3)
(128, 128, 3)


In [50]:
# 3. Initialize & build the model (10 points)

from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Convolution2D(32,kernel_size=(3,3), activation ='relu', input_shape = (128, 128, 3))) 
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Dropout(0.8))

model.add(Convolution2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.8))

# model.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# #model.add(Dropout(0.8))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128,activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 12, activation = 'softmax')) 

In [0]:
# 4. Optimize the model (8 points)

from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

#To use adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.001)
#Set the loss function and optimizer for the model training
model.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

In [61]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 31, 31, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 128)      

In [63]:
# 5. Predict the accuracy for both train and validation data (7 points)

history = model.fit(x_train_set,y_train_set,
                    epochs=10, 
                    validation_data=(x_test,y_test),
                    verbose = 1,
                    initial_epoch=0)

Train on 3325 samples, validate on 1425 samples
Epoch 1/10
3325/3325 [==============================] - 140s 42ms/step - loss: 2.2730 - acc: 0.2108 - val_loss: 1.9635 - val_acc: 0.3628
Epoch 2/10
3325/3325 [==============================] - 135s 41ms/step - loss: 1.8869 - acc: 0.3408 - val_loss: 1.6617 - val_acc: 0.4091
Epoch 3/10
3325/3325 [==============================] - 135s 41ms/step - loss: 1.6367 - acc: 0.4316 - val_loss: 1.3586 - val_acc: 0.5425
Epoch 4/10
3325/3325 [==============================] - 135s 41ms/step - loss: 1.4032 - acc: 0.5053 - val_loss: 1.1766 - val_acc: 0.5979
Epoch 5/10
3325/3325 [==============================] - 135s 41ms/step - loss: 1.1519 - acc: 0.6048 - val_loss: 1.1329 - val_acc: 0.6316
Epoch 6/10
3325/3325 [==============================] - 134s 40ms/step - loss: 0.9766 - acc: 0.6562 - val_loss: 1.0574 - val_acc: 0.6611
Epoch 7/10
3325/3325 [==============================] - 135s 41ms/step - loss: 0.9046 - acc: 0.6902 - val_loss: 0.9486 - val_acc: 